In [1]:
import pandas as pd
rway_img_df_ok = pd.read_csv("collection_images_download.csv")

In [2]:
rway_img_df = rway_img_df_ok.copy()

# EXTRACT BRAND FROM IMAGE NAME
# Creazione delle nuove colonne
rway_img_df['brand'] = rway_img_df['image_name'].str.lower()
rway_img_df['brand'] = rway_img_df['brand'].str.split('couture').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.split('ready-to-wear').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.split('menswear').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.replace("-", " ")
rway_img_df['brand'] = rway_img_df['brand'].str.strip()

In [20]:
# create a small dataframe with the brand in scope
brand_in_scope = ['versace', 'hermes']
rway_img_df_small = pd.DataFrame()

for i,row in enumerate(rway_img_df['brand']):
    if row in brand_in_scope:
        rway_img_df_small = rway_img_df_small.append(rway_img_df.iloc[i])
        
rway_img_df_small['image_path_no_bkgnd'] = rway_img_df_small['image_path'].astype(str)

# Execute 
# for each row in rway_img_df_small['image_path_no_bkgnd'] change the output path
for index, row in rway_img_df_small.iterrows():
    # Split the image path into base name and extension
    base_name, extension = row['image_path_no_bkgnd'].split('.')

    # Add "-no-bkgnd" to the base name
    new_base_name = f"{base_name}-no-bkgnd"

    # Concatenate the new base name with the extension
    new_image_path = f"{new_base_name}.{extension}"

    # Replace the original image path with the new image path
    rway_img_df_small.at[index, 'image_path_no_bkgnd'] = new_image_path


In [22]:

#! it's takes 1h for 1694 imgs (% of full sample size)
#! Time estimate per img 2secs

from rembg import remove
import os

path_prefix = "./imgs/images/images/"

# Assuming rway_img_df_small is a pandas DataFrame containing the image paths
# with a column named 'image_path_no_bkgnd'

for index, row in rway_img_df_small.iterrows():
    input_path = f"{path_prefix}{row['image_path']}"
    output_path = f"{path_prefix}{row['image_path_no_bkgnd']}"

    with open(input_path, 'rb') as i:
        with open(output_path, 'wb') as o:
            input_data = i.read()
            output_data = remove(input_data)
            o.write(output_data)


In [ ]:
(1694/132153)*100

In [ ]:
# Display the updated DataFrame
print(way_img_df_small)

In [5]:
# from small dataframe create train (90%) and test (10%) set
from sklearn.model_selection import train_test_split
train, test = train_test_split(rway_img_df_small, test_size=0.1, random_state=42)
